In [1]:
import pandas as pd
import recordlinkage
df_with_truth = pd.read_csv('restaurant.csv', skip_blank_lines=True)
# print(len(df_with_truth))
# df_with_truth
df = df_with_truth.drop(columns=['cluster', 'phone','type'])
# df.head(2)
import re
# print("Remove irrelevant separators:")
irrelevant_regex = re.compile(r'[^a-z0-9\s]')
# print("Remove multi-spaces:")
multispace_regex = re.compile(r'\s\s+')
def assign_no_symbols_name(df):
    return df.assign(
        name=df['name']
             .str.replace(irrelevant_regex, ' ')
             .str.replace(multispace_regex, ' '))

df = assign_no_symbols_name(df)
df.head(5)
import recordlinkage as rl
from recordlinkage.index import Full

indexer = Full()
pairs = indexer.index(df)

print(f"Full index: {len(df)} records, {len(pairs)} pairs")
compare_cl = recordlinkage.Compare()
compare_cl.string('name', 'name', method='jarowinkler',label='name')
compare_cl.string('city', 'city', method='jarowinkler',label='city')
compare_cl.string('addr', 'addr', method='jarowinkler',label='addr')
comparision_vectors = compare_cl.compute(pairs, df)
# comparision_vectors.head(5)
import numpy as np
scores=np.average(comparision_vectors.values,axis=1)
scored_comparision_vectors=comparision_vectors.assign(score=scores)
scored_comparision_vectors.head()
matches=scored_comparision_vectors[scored_comparision_vectors['score']>=0.9]
ud={}
unique=[]
duplicate=[]
for i in set(matches.index):
    if (list(i)[0]) not in ud:
        unique.append((list(i)[0]))
        ud=list(ud)+unique
                    
    if (list(i)[1]) not in ud:
        duplicate.append((list(i)[1]))
        ud=list(ud)+duplicate
    ud=set(ud)
unique=np.array(unique)
unique.sort()
# print(f"Unique Records index:",unique)
duplicate=np.array(duplicate)
duplicate.sort()
# print(f"Duplicate Records index:",duplicate)  
ud=list(ud)
print(len(unique))
print(len(duplicate))
print(len(ud))
# print(ud)
unique1=set(unique)
unique2=set(df.index)-set(ud)
unique=list(unique1)+list(unique2)
uniq=df_with_truth.iloc[unique]
uniq.to_excel("Rest Unique.xlsx")
dup=df_with_truth.iloc[duplicate]
dup.to_excel("Rest duplicate.xlsx")
print(f"Total Number Records:",len(uniq)+len(dup))
input("Press the enter for stop the program")

Full index: 881 records, 387640 pairs
103
108
211
Total Number Records: 881
Press the enter for stop the programk


'k'